In [58]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
from io import StringIO
from typing import Dict, List, Tuple
import json



def pfr_web(letter: str) -> str:
  assert(len(letter) > 0)
  return f"https://www.pro-football-reference.com/players/{letter[0].upper()}/"

In [48]:
res = requests.get(pfr_web('A'))
html = BeautifulSoup(res.text)
links = html.find("div", {"id": "all_players"}).find_all("a")

player_name = links[0].text
player_id = links[0].get('href').split('/')[-1].split('.')[0]


In [49]:
type(links[0])

bs4.element.Tag

In [50]:
def map_ids() -> Tuple[Dict[str, str], Dict[str, str]]:
  name_to_id = {}
  id_to_name = {}
  for i in range(0, 26):
    letter = chr(ord('a') + i)
    res = requests.get(pfr_web(letter.upper()))
    html = BeautifulSoup(res.text)
    links: List[Tag] = html.find("div", {"id": "all_players"}).find_all("a")
    for link in links:
      player_name = link.text
      player_id = link.get('href').split('/')[-1].split('.')[0]
      name_to_id[player_name] = player_id
      id_to_name[player_id] = player_name
  return name_to_id, id_to_name

In [42]:
name_to_id, id_to_name = map_ids()

In [59]:
test = json.dumps(name_to_id)

In [66]:
json.loads(test)["Dak Prescott"]

'PresDa01'